In [56]:
from sklearn.model_selection import ParameterSampler
import numpy as np
from pathlib import Path
from copy import deepcopy
import json

In [61]:
sampler = ParameterSampler(
    param_distributions={'num_samples':np.linspace(10, 200, 39),
                         'temperature':np.linspace(1, 5, 30)},
    n_iter = 1170
)

In [62]:
clf_path = "notebook/morozov/Sberbank_project/adversarial_sber/logs/age/target_clf/171151-1409-gru_with_amounts/model.tar.gz"
MASKED_LM_PATH = "./presets/age/lm.model.tar.gz"
CLF_PATH = "/notebook/morozov/Sberbank_project/adversarial_sber/logs/age/target_clf/171151-1409-gru_with_amounts/model.tar.gz"
DATA_PATH = "/notebook/morozov/data/age/test.jsonl"
output_path = "./results/brute_force"

In [63]:
base_config = {
  "data_path": DATA_PATH,
  "output_path": output_path,
  "attacker": {
    "type": "sampling_fool",
    "masked_lm": {
      "type": "from_archive",
      "archive_file": MASKED_LM_PATH
    },
    "classifier": {
      "type": "from_archive",
      "archive_file": CLF_PATH
    },
    "reader": {
      "type": "from_archive",
      "archive_file": CLF_PATH
    },
    "num_samples": None,
    "temperature": None,
    "device": 0
  }
}

In [64]:
command = """PYTHONPATH=. python advsber/commands/attack.py \
             {CONFIG_PATH} \
             --samples 500"""

In [67]:
with open('brute_force_sf_gru_age.sh', 'w') as w:
    for i, params in enumerate(sampler):
        exp_path = f"results/brute_force/dist/exp_{i}"
        Path(exp_path).mkdir(parents=True, exist_ok=True)
        conf_path = f"results/brute_force/dist/exp_{i}/config.json"
        config = deepcopy(base_config)
        
        for key in params.keys():
            config["attacker"][key] = params[key]
        
        with open(conf_path, 'w') as f:
            json.dump(config, f)
        
        w.write(command.format(EXP_PATH=exp_path, CONFIG_PATH=conf_path))
        w.write('\n')